<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Mixed_Integer_Problem_for_Draft_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1  Initialisierung und Konfiguration

## 1.1 Bibliotheken installieren und laden


In [1]:
try:
    import mip
except ImportError:
    import sys
    !{sys.executable} -m pip install mip
import pandas as pd
import numpy as np
import re
import random
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


## 1.2 Liga-Parameter und Rahmenbedingungen definieren

In [2]:
year = 2024
num_teams = 12
num_rounds = 15
num_weeks = 17
allowed_positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST'}
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 1, 'K': 1, 'DST': 1}

# 2 Datenbeschaffung und -vorverarbeitung


## 2.1 ADP, Bye Weeks und Projections laden

In [12]:
adp_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/pre_season_data/adp_projections_{year}.csv'
player_adp = (
    pd.read_csv(adp_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'adp': 'ADP', 'points': 'TTL'})
      .fillna({'ADP': 999})
      .loc[:, ['Player', 'POS', 'ADP', 'TTL']]
      .sort_values('ADP')
      .reset_index(drop=True)
)


bye_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/bye_weeks/bye_weeks_{year}.csv'
bye_weeks = pd.read_csv(bye_url)


season_projection_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/pre_season_data/adp_projections_{year}.csv'
season_projections = (
    pd.read_csv(season_projection_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'team': 'Team', 'points': 'TTL', 'adp': 'ADP'})
      .loc[:, ['Player', 'POS', 'Team', 'TTL', 'ADP']]
      .merge(bye_weeks[['Abbreviation', 'Bye']], left_on='Team', right_on='Abbreviation', how='left')
      .drop(columns='Abbreviation')
      .drop_duplicates(subset='Player', keep='first')
)
season_projections['avg_proj'] = season_projections['TTL'] / np.where(
    season_projections['Bye'].isna(),
    num_weeks,
    num_weeks - 1
)
week_cols = [f'Week_{w}' for w in range(1, num_weeks + 1)]
for week in week_cols:
    season_projections[week] = np.where(
        season_projections['Bye'] == week,
        0,
        season_projections['avg_proj']
    )
season_projections = season_projections.drop(columns='avg_proj')

real_projections_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/real_projections/real_projections_{year}.csv'
real_projections = (
    pd.read_csv(real_projections_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'points': 'Projection', 'week': 'Week'})
      .loc[:, ['Player', 'POS', 'Projection', 'Week']]
      .pivot_table(
         index=['Player', 'POS'],
         columns='Week',
         values='Projection',
         aggfunc='first'
      )
      .rename_axis(None, axis=1)
      .add_prefix('Week_')
      .reset_index()
      .fillna(0)
)

## 2.2 Berechnung von TTL, Dropoff und VOR

In [4]:
dropoff_w = {'QB':1.0,'RB':1.0,'WR':1.0,'TE':0.9,'K':0.4,'DST':0.3}
vor_w     = {'QB':0.8,'RB':1.0,'WR':1.0,'TE':0.8,'K':0.25,'DST':0.25}

rep_ttl = (
    season_projections
      .groupby('POS')['TTL']
      .apply(lambda s: s.nlargest(lineup_req[s.name]*num_teams)
                   .iloc[-1]
             if len(s) >= lineup_req[s.name]*num_teams else 0)
)

season_projections = (
    season_projections
      .sort_values(['POS','TTL'], ascending=[True,False])
      .assign(
          dropoff=lambda df: (
              df.groupby('POS')['TTL']
                .diff(-1)
                .fillna(0)
                .mul(df['POS'].map(dropoff_w))
          ),
          VOR=lambda df: (
              (df['TTL'] - df['POS'].map(rep_ttl))
                .clip(lower=0)
                .mul(df['POS'].map(vor_w))
          )
      )
      .fillna({'dropoff': 0, 'VOR': 0, 'ADP': 999})
      .sort_values(['ADP', 'TTL'], ascending=[True, False])
      .reset_index(drop=True)
)


# 3 Modellparameter und Variablenaufbau

## 3.1 Spielerlisten und Parameter-Mapping

In [5]:
# 1. INITIALIZATION
players            = (season_projections[['Player', 'POS', 'ADP', 'TTL', 'dropoff', 'VOR']]
                      .copy()
                      .sort_values(['ADP', 'TTL'], ascending=[True, False])
                      .reset_index(drop=True))
players['Rank']    = players.index + 1

players_list       = players['Player'].copy().tolist()
positions          = {'QB', 'RB', 'WR', 'TE', 'K', 'DST', 'FLEX'}
flex_eligible      = {'RB', 'WR', 'TE'}
weeks              = list(range(1, 18))

pos                = dict(zip(season_projections['Player'], season_projections['POS']))
lineup_req         = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
max_req            = {'QB': 2, 'RB': 999, 'WR': 999, 'TE': 2, 'K': 1, 'DST': 1}
week_cols          = [col for col in season_projections.columns if col.startswith('Week_')]

season_projections_dict = {
	row['Player']: {
		**{int(week.replace('Week_', '')): row[week] for week in week_cols},
		'dropoff': row['dropoff'],
		'VOR':     row['VOR']
	}
	for _, row in season_projections.iterrows()
}

beta               = {t: 120.0 for t in weeks}
alpha              = 0.3
lambda_0           = 1
lambda_1           = 3
lambda_2           = 4
lambda_3           = 0.5
lambda_4           = 50
df_sorted          = players.sort_values('Rank').reset_index(drop=True)
topk_pct           = 0.0025

# 2. DRAFT ORDER SETUP

teams              = [f'Team {i+1}' for i in range(num_teams)]
DM_team            = 'Team 1'

draft_order        = []
for rnd in range(num_rounds):
	order = teams if rnd % 2 == 0 else teams[::-1]
	draft_order += order

# 3. OPPONENT PICK FUNCTION

def opponent_pick(roster, available, Rk, lineup_req, topk_pct=topk_pct):
	remaining_players = sorted(available, key=lambda p: Rk[p])
	topk              = max(1, int(len(remaining_players) * topk_pct))

	deficits = {
		j: lineup_req[j] - sum(1 for p in roster if pos[p] == j)
		for j in lineup_req if j != 'FLEX'
	}
	needed = [j for j, d in deficits.items() if d > 0]

	if needed:
		candidates = [p for p in remaining_players if pos[p] in needed]
		pool       = candidates[:topk] if len(candidates) >= topk else candidates
		if pool:
			return random.choice(pool)

	return random.choice(remaining_players[:topk])

# 4. DRAFT INITIALIZATION

rosters     = {tm: [] for tm in teams}
available   = set(players_list)
draft_log   = []




## 3.2 Optimierungsvariablen und Zielfunktion konfigurieren

In [6]:
# 1. INITIALIZATION
players            = (season_projections[['Player', 'POS', 'ADP', 'TTL', 'dropoff', 'VOR']]
                      .copy()
                      .sort_values(['ADP', 'TTL'], ascending=[True, False])
                      .reset_index(drop=True))
players['Rank']    = players.index + 1

players_list       = players['Player'].copy().tolist()
positions          = {'QB', 'RB', 'WR', 'TE', 'K', 'DST', 'FLEX'}
flex_eligible      = {'RB', 'WR', 'TE'}
weeks              = list(range(1, 18))

pos                = dict(zip(season_projections['Player'], season_projections['POS']))
lineup_req         = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
max_req            = {'QB': 2, 'RB': 999, 'WR': 999, 'TE': 2, 'K': 1, 'DST': 1}
week_cols          = [col for col in season_projections.columns if col.startswith('Week_')]

season_projections_dict = {
	row['Player']: {
		**{int(week.replace('Week_', '')): row[week] for week in week_cols},
		'dropoff': row['dropoff'],
		'VOR':     row['VOR']
	}
	for _, row in season_projections.iterrows()
}

beta               = {t: 120.0 for t in weeks}
alpha              = 0.3
lambda_0           = 1
lambda_1           = 3
lambda_2           = 4
lambda_3           = 0.5
lambda_4           = 40
df_sorted          = players.sort_values('Rank').reset_index(drop=True)
topk_pct           = 0.0025

# 2. DRAFT ORDER SETUP

teams              = [f'Team {i+1}' for i in range(num_teams)]
DM_team            = 'Team 1'

draft_order        = []
for rnd in range(num_rounds):
	order = teams if rnd % 2 == 0 else teams[::-1]
	draft_order += order

# 3. OPPONENT PICK FUNCTION

def opponent_pick(roster, available, Rk, lineup_req, topk_pct=topk_pct):
	remaining_players = sorted(available, key=lambda p: Rk[p])
	topk              = max(1, int(len(remaining_players) * topk_pct))

	deficits = {
		j: lineup_req[j] - sum(1 for p in roster if pos[p] == j)
		for j in lineup_req if j != 'FLEX'
	}
	needed = [j for j, d in deficits.items() if d > 0]

	if needed:
		candidates = [p for p in remaining_players if pos[p] in needed]
		pool       = candidates[:topk] if len(candidates) >= topk else candidates
		if pool:
			return random.choice(pool)

	return random.choice(remaining_players[:topk])

# 4. DRAFT INITIALIZATION

rosters     = {tm: [] for tm in teams}
available   = set(players_list)
draft_log   = []


# 5. MAIN DRAFT LOOP

for pick_idx, team in enumerate(draft_order, start=1):

    for p in available:
        ranks = df_sorted.loc[df_sorted.Player == p, 'Rank']
        if len(ranks) != 1:
            print(f'Problem bei Spieler {p}: Anzahl gefundener Ränge = {len(ranks)}')

    remaining_players = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player == p, 'Rank'].item())
    Rk                = {p: i+1 for i, p in enumerate(remaining_players)}
    player_vars       = set(remaining_players) | set(rosters[team])
    picks_remaining   = num_rounds - len(rosters[team])

    if team == DM_team:
        m = Model(sense=maximize, solver_name='CBC')

        y = {i: m.add_var(var_type=BINARY, name=f'y_{i}') for i in remaining_players}
        x = {(i, t): m.add_var(var_type=CONTINUOUS, name=f'x_{i}_{t}') for i in remaining_players for t in weeks}
        z = {t: m.add_var(var_type=BINARY, name=f'z_{t}') for t in weeks}

        m.objective = (
            lambda_0 * xsum(season_projections_dict[i][t] * x[i, t] for i in remaining_players for t in weeks)
            + lambda_1 * xsum(z[t] for t in weeks[:15])
            + lambda_2 * xsum(z[t] for t in weeks[15:])
            + lambda_3 * xsum(season_projections_dict[i]['dropoff'] * y[i] for i in remaining_players)
            - lambda_4 * xsum(season_projections_dict[i]['VOR'] * y[i] for i in remaining_players)
        )

        # for p in rosters[team]:
        #     m += y[p] == 1
        m += xsum(y[i] for i in remaining_players) == picks_remaining

        for pos_name, req in lineup_req.items():
            if pos_name != 'FLEX':
                already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
                need = max(0, req - already_satisfied)
                m += xsum(y[i] for i in remaining_players if pos[i] == pos_name) >= need

        for t in weeks:
            for j in positions - {'FLEX'}:
                m += xsum(y[i] for i in remaining_players if pos[i] == j) <= max_req[j]
            for i in remaining_players:
                m += (
                    (x[i, t] if pos[i] in positions - {'FLEX'} else 0) +
                    (x[i, t] if pos[i] in flex_eligible else 0)
                    <= 1
                )
        for i in remaining_players:
            for t in weeks:
                m += x[i, t] <= y[i]
        for t in weeks:
            m += z[t] <= xsum(season_projections_dict[i][t] * x[i, t] for i in remaining_players) / beta[t]

        n_k = pick_idx
        for future_pick in range(pick_idx + 1, pick_idx + picks_remaining * len(teams), len(teams)):
            top_cut = int(alpha * (future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i, r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ((future_pick - n_k) // len(teams))

        m.optimize()
        if m.num_solutions == 0:
            raise RuntimeError(f'No feasible solution at pick {pick_idx}. Check constraints and player pool.')

        print(f"\n--- DM Pick {pick_idx} ---")
        for i in remaining_players:
            if y[i].x is not None and y[i].x >= 0.9:
                ttl = sum(season_projections_dict[i][t] for t in weeks)
                vor = season_projections_dict[i]['VOR']
                print(f"{i:25} ({pos[i]})  TTL={ttl:6.1f}   VOR={vor:6.1f}")

        chosen = [i for i in remaining_players if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f'No feasible pick at {pick_idx}')
        pick = min(chosen, key=lambda i: Rk[i])

    else:
        current_round   = (pick_idx - 1) // len(teams) + 1
        dynamic_topk_pct = min(current_round * 2 * topk_pct, 1.0)
        pick = opponent_pick(
            roster    = rosters[team],
            available = available,
            Rk        = Rk,
            lineup_req= lineup_req,
            topk_pct  = dynamic_topk_pct
        )

    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        'Pick':    pick_idx,
        'Team':    team,
        'Player':  pick,
        'Round':   (pick_idx - 1) // len(teams) + 1,
        'POS':     pos[pick]
    })

# 6. CREATE DRAFT DATAFRAME

df_draft = pd.DataFrame(draft_log)
print(m.status)
#print(m.num_constrs, m.num_vars)
print(m)



--- DM Pick 1 ---
Christian McCaffrey       (RB)  TTL= 353.0   VOR= 162.0
Breece Hall               (RB)  TTL= 309.0   VOR= 118.0
Justin Jefferson          (WR)  TTL= 308.0   VOR=  79.0
Puka Nacua                (WR)  TTL= 287.0   VOR=  58.0
Kyren Williams            (RB)  TTL= 254.0   VOR=  63.0
Josh Allen                (QB)  TTL= 371.0   VOR=  63.2
Josh Jacobs               (RB)  TTL= 243.0   VOR=  52.0
Jalen Hurts               (QB)  TTL= 365.0   VOR=  58.4
Rachaad White             (RB)  TTL= 242.0   VOR=  51.0
Sam LaPorta               (TE)  TTL= 227.0   VOR=  56.8
Joe Mixon                 (RB)  TTL= 229.0   VOR=  38.0
Alvin Kamara              (RB)  TTL= 241.0   VOR=  50.0
Trey McBride              (TE)  TTL= 208.0   VOR=  41.6
Brandon Aubrey            (K)  TTL= 163.0   VOR=   4.8
Jets                      (DST)  TTL= 100.0   VOR=   2.1

--- DM Pick 24 ---
Josh Jacobs               (RB)  TTL= 243.0   VOR=  52.0
Jalen Hurts               (QB)  TTL= 365.0   VOR=  58.4
Rachaad W

# 5 Ergebnisaufbereitung

## 5.1 Draft-Log exportieren und zusammenführen

In [7]:
# Gesamtprojektion pro Team
team_ttl_proj = (
    df_draft
    .merge(season_projections[['Player','TTL']], on='Player', how='left')
    .assign(TTL=lambda df: df['TTL'].fillna(0))
    .groupby('Team')['TTL']
    .sum()
    .reset_index(name='TTL_proj')
)
print(team_ttl_proj)

# Positionen zählen
position_counts = df_draft.pivot_table(
    index='Team', columns='POS', aggfunc='size', fill_value=0
)
print(position_counts)

# Draft-Infos pro Team
result_dfs = {
    team: (
        df_draft[df_draft['Team']==team]
        .sort_values('Pick')
        .assign(Pick_Info=lambda df:
            'Round '+df['Round'].astype(str)+' Pick '+df['Pick'].astype(str))
        [['Player','Pick_Info','POS']]
    )
    for team in df_draft['Team'].unique()
}

for team, df in result_dfs.items():
    print(f'=== {team} ===')
    print(df)
    print()


       Team  TTL_proj
0    Team 1    3409.8
1   Team 10    3218.8
2   Team 11    3047.0
3   Team 12    2876.0
4    Team 2    2950.3
5    Team 3    3108.1
6    Team 4    2964.4
7    Team 5    3041.3
8    Team 6    2987.5
9    Team 7    2848.2
10   Team 8    3055.3
11   Team 9    2812.0
POS      DST  K  QB  RB  TE  WR
Team                           
Team 1     1  1   3   3   1   6
Team 10    1  1   3   2   1   7
Team 11    1  1   2   5   1   5
Team 12    2  1   2   5   1   4
Team 2     2  1   1   3   1   7
Team 3     1  1   3   5   3   2
Team 4     1  1   2   5   1   5
Team 5     1  1   1   4   1   7
Team 6     1  1   2   6   2   3
Team 7     1  1   1   4   2   6
Team 8     1  1   3   4   2   4
Team 9     1  2   1   6   1   4
=== Team 1 ===
                  Player          Pick_Info  POS
0    Christian McCaffrey     Round 1 Pick 1   RB
23           Josh Jacobs    Round 2 Pick 24   RB
24           Jalen Hurts    Round 3 Pick 25   QB
47          Deebo Samuel    Round 4 Pick 48   WR
48    

## 5.2 Ergebnis-Statustext ausgeben

# 6 Evaluation

In [14]:
roster_projections = (
    df_draft
      .merge(
         real_projections.drop(columns=['POS']),
         on='Player',
         how='left'
      )
      .sort_values(['Team','Pick'])
      .sort_values(by=['Team', 'Pick'])
      .reset_index(drop=True)
)
roster_projections = roster_projections
roster_projections

,Pick,Team,Player,Round,POS,Week_1,Week_2,Week_3,Week_4,Week_5,...,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17
0,1,Team 1,Christian McCaffrey,1,RB,21.40,0.00,0.00,0.00,0.00,...,0.00,0.00,19.20,22.30,19.80,19.00,0.00,0.00,0.00,0.00
1,24,Team 1,Josh Jacobs,2,RB,14.30,13.80,14.90,14.10,13.10,...,13.80,14.00,0.00,14.00,15.80,16.80,16.40,17.50,19.70,18.60
2,25,Team 1,Jalen Hurts,3,QB,21.80,22.70,21.20,20.60,0.00,...,19.70,22.40,22.00,21.80,21.50,21.50,22.50,20.70,21.90,0.00
3,48,Team 1,Deebo Samuel,4,WR,13.70,16.20,0.00,12.40,14.90,...,15.00,0.00,14.90,14.80,12.10,12.10,11.80,11.80,11.20,13.30
4,49,Team 1,Trey McBride,5,TE,12.10,12.80,12.90,0.00,11.50,...,11.60,12.40,12.10,0.00,12.70,13.60,14.30,14.70,15.90,14.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,129,Team 9,Jerome Ford,11,RB,12.20,13.40,12.30,11.80,12.10,...,0.00,6.48,0.00,6.56,6.95,6.34,7.32,8.08,12.50,12.50
177,136,Team 9,Jonathon Brooks,12,RB,0.00,0.00,0.00,0.00,4.10,...,4.72,4.80,0.00,0.00,5.07,4.29,5.29,0.00,0.00,0.00
178,153,Team 9,Tyler Allgeier,13,RB,6.72,6.00,6.44,6.34,7.13,...,7.39,7.78,7.40,6.35,0.00,6.19,5.32,5.73,5.90,5.41
179,160,Team 9,Jaleel McLaughlin,14,RB,7.43,8.87,7.43,6.53,6.75,...,6.02,5.39,4.95,4.34,4.04,5.15,0.00,6.69,0.00,6.71
